Day started collecting data Aug 11 2020

### Explanation:

* Inorder to get the matches of a specific season we need to do the following:
1. get the season code (1 request). 
2. get the basic details of all matches (i.e. Fixtures) (1 request for all matches)
3. get the lineups and more information about a match (1 request per match)
4. get the players ratings of each match (1 request per match)

Things to note:
1. We must query the fixtures corrected to a specified timezone
2. We must find a way to update matches that occur while working on the project.
3. Everything must be automated eventually

In [148]:
hel

NameError: name 'hel' is not defined

In [74]:
import requests
import pprint
import re

import pandas as pd
import numpy as np
import time

In [149]:
# I change it before every commit
my_api_key = "2b7b92f240msh6817a320b35289bp1b47d2jsnf9a630fd5bc8"

Getting the codes for the seasons 2018, 19, 20

In [18]:

url = "https://api-football-v1.p.rapidapi.com/v2/leagues/country/SA"

headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': my_api_key
    }

response = requests.request("GET", url, headers=headers)



In [19]:
pprint.pprint(response.json())

{'api': {'leagues': [{'country': 'Saudi-Arabia',
                      'country_code': 'SA',
                      'coverage': {'fixtures': {'events': True,
                                                'lineups': True,
                                                'players_statistics': True,
                                                'statistics': True},
                                   'odds': False,
                                   'players': True,
                                   'predictions': True,
                                   'standings': True,
                                   'topScorers': True},
                      'flag': 'https://media.api-sports.io/flags/sa.svg',
                      'is_current': 0,
                      'league_id': 419,
                      'logo': 'https://media.api-sports.io/football/leagues/307.png',
                      'name': 'Pro League',
                      'season': 2018,
                      'season_end': '2019-05

In [72]:
season2018code = 419
season2019code = 901
season2020code = 2984


getting the code for ٍRiyadh's timezone so that the date variable is relative

In [24]:
url = "https://api-football-v1.p.rapidapi.com/v2/timezone"

headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': my_api_key
    }

response = requests.request("GET", url, headers=headers)

pprint.pprint(response.json())



{'api': {'results': 425,
         'timezone': ['Africa/Abidjan',
                      'Africa/Accra',
                      'Africa/Addis_Ababa',
                      'Africa/Algiers',
                      'Africa/Asmara',
                      'Africa/Bamako',
                      'Africa/Bangui',
                      'Africa/Banjul',
                      'Africa/Bissau',
                      'Africa/Blantyre',
                      'Africa/Brazzaville',
                      'Africa/Bujumbura',
                      'Africa/Cairo',
                      'Africa/Casablanca',
                      'Africa/Ceuta',
                      'Africa/Conakry',
                      'Africa/Dakar',
                      'Africa/Dar_es_Salaam',
                      'Africa/Djibouti',
                      'Africa/Douala',
                      'Africa/El_Aaiun',
                      'Africa/Freetown',
                      'Africa/Gaborone',
                      'Africa/Harare',
      

requesting the matches data corrected to our timezone

In [46]:
def get_fixtures(seasonCode):
    """Returns all season fixtures as a json type object.

    Args:
    SeasonCode: int or String. Which is the code specified by the
                    api for querying the details of a specific season 


    Returns:
    fixtures: which is a json object that contains all fixtures of the desired season.
    """
    url = f"https://api-football-v1.p.rapidapi.com/v2/fixtures/league/{seasonCode}?timezone=Asia/Riyadh"

    headers = {
        'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
        'x-rapidapi-key': my_api_key
        }

    fixtures = requests.request("GET", url, headers=headers)

    return fixtures.json()["api"]["fixtures"]

In [112]:
len(season2020.columns)

17

In [109]:
def addMatchInfo(season_df, year, match_index, match_info):
    fixture_id = match_info["fixture_id"]
    home_team = match_info["homeTeam"]["team_name"]
    away_team = match_info["awayTeam"]["team_name"]
    referee = match_info["referee"]
    nRound = match_info["round"].split("-")[1].replace(" ", "")
    date = match_info["event_date"]
    venue = match_info["venue"]
    home_goals = match_info["goalsHomeTeam"]
    away_goals = match_info["goalsAwayTeam"]
    status = match_info["status"]

    season_df.loc[match_index] = [fixture_id, home_team,
                         away_team, home_goals,
                         away_goals, venue,
                         referee, nRound,
                         date, year, status,
                        np.nan, np.nan, np.nan, #These columns require a request per match,
                        np.nan, np.nan, np.nan,
                                 np.nan, np.nan]
    return season_df

In [101]:
def create_df(fixtures, year):
    """Creates a dataframe with the the basic details of the match.

    Args:
    fixtures: Json object. that contains all fixtures, the one resulted from the get_fixtures() function
    
    year: String. Which specifies which season are we creating the dataframe for (e.g. 2020/2021)

    Returns:
    season_df: Pandas Dataframe. that is consisted of all matches 'basic' details.
    """
    matches = fixtures["api"]["fixtures"]
    
    season_df = pd.DataFrame(columns=['fixture_id', 'home_team', 'away_team', 'home_goals',
                                      'away_goals', 'venue', 'referee',
                                      'round', 'date', 'season','status',
                                     "home_coach", "home_foramtion", "home_players",
                                    "away_coach", "away_formation", "away_players",
                                     "H_avg_ratings", "A_avg_ratings"])
    
    for i in range(240):
        season_df = addMatchInfo(season_df, year, i, matches[i])
        
    return season_df

In [130]:
def addMatchDetails(season_df, match_index):
    """Adds all the extra information of finished matches to the dataframe.

    Args:
    season_df: Pandas Dataframe. that is consisted of all matches 'basic' details.
    
    match_index: index of the match to be updated


    Returns:
    season_df: Pandas Dataframe. The same dataframe provided but with added 
                                features to the finished matches
    """
    

    match = season_df.loc[match_index]
    
    url = f"https://api-football-v1.p.rapidapi.com/v2/lineups/{match['fixture_id']}"

    headers = {
        'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
        'x-rapidapi-key': my_api_key
        }

    response = requests.request("GET", url, headers=headers).json()

    #home team data

    home_coach = response["api"]["lineUps"][match["home_team"]]["coach"]
    home_formation = response["api"]["lineUps"][match["home_team"]]["formation"]

    home_players = []
    for player in response["api"]["lineUps"][match["home_team"]]["startXI"]:
        home_players.append((player["player"], player["pos"]))


    #away team data                                             
    away_coach = response["api"]["lineUps"][match["away_team"]]["coach"]
    away_formation = response["api"]["lineUps"][match["away_team"]]["formation"]

    away_players = []
    for player in response["api"]["lineUps"][match["away_team"]]["startXI"]:
        away_players.append((player["player"], player["pos"]))                                            

    season_df.loc[match_index, ["home_coach", "home_foramtion",
                  "home_players", "away_coach",
                  "away_formation", "away_players"]] = [home_coach, home_formation, 
                                                       home_players, away_coach,
                                                       away_formation, away_players]
    
    return season_df

In [92]:
season2020.columns

Index(['fixture_id', 'home_team', 'away_team', 'home_goals', 'away_goals',
       'venue', 'referee', 'round', 'date', 'season', 'status', 'home_coach',
       'home_foramtion', 'home_players', 'away_coach', 'away_formation',
       'away_players'],
      dtype='object')

In [93]:
def update_match(match, season_df):
    
    match_index = season_df.loc[season_df["fixture_id"] == match["fixture_id"]].index[0]
    
    season_df = addMatchInfo(season_df, season_df.season.unique()[0], match_index, match)

    season_df = addMatchDetails(season_df, match_index)
    return season_df

In [107]:
#The following code was used to extract matches that happend while I was working on the project
def update_df(season_df, season_code):

    not_started_matches = season_df.query("status == 'Not Started'").index

    all_matches = get_fixtures(season_code)
    
    for i in not_started_matches[:10]:
        match = all_matches[i]
        if match["status"] == "Match Finished":
            season_df = update_match(match, season_df)
        
    return season_df


-----

In [78]:
season2020_fixtures = get_fixtures(season2020code)

Now we should have the data for all matches in the 2018/2019 season.
We have 16 teams playing 30 rounds so we got 16 * 30 / 2 = 240 matches

In [15]:
len(season2020_fixtures)

240

Checking how each match is structured

In [16]:
season2020_fixtures[0]

{'fixture_id': 635867,
 'league_id': 2984,
 'league': {'name': 'Pro League',
  'country': 'Saudi-Arabia',
  'logo': 'https://media.api-sports.io/football/leagues/307.png',
  'flag': 'https://media.api-sports.io/flags/sa.svg'},
 'event_date': '2020-10-17T18:00:00+03:00',
 'event_timestamp': 1602946800,
 'firstHalfStart': 1602946800,
 'secondHalfStart': 1602950400,
 'round': 'Regular Season - 1',
 'status': 'Match Finished',
 'statusShort': 'FT',
 'elapsed': 90,
 'venue': 'Prince Faisal bin Fahd Stadium',
 'referee': 'Mohammed Al Hoaish',
 'homeTeam': {'team_id': 2940,
  'team_name': 'Al Shabab',
  'logo': 'https://media.api-sports.io/football/teams/2940.png'},
 'awayTeam': {'team_id': 2951,
  'team_name': 'Abha',
  'logo': 'https://media.api-sports.io/football/teams/2951.png'},
 'goalsHomeTeam': 1,
 'goalsAwayTeam': 0,
 'score': {'halftime': '1-0',
  'fulltime': '1-0',
  'extratime': None,
  'penalty': None}}

In [69]:
season2020 = create_df(season2020_fixtures, "2020/2021")

In [70]:
season2020

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,round,date,season,status,home_coach,home_foramtion,home_players,away_coach,away_formation,away_players
0,635867,Al Shabab,Abha,1,0,Prince Faisal bin Fahd Stadium,Mohammed Al Hoaish,1,2020-10-17T18:00:00+03:00,2020/2021,Match Finished,NaN,NaN,NaN,NaN,NaN,NaN
1,635868,Al Taawon,Al-Faisaly FC,1,1,King Abdullah bin Abdulaziz City,Faisal Alblwi,1,2020-10-17T18:10:00+03:00,2020/2021,Match Finished,NaN,NaN,NaN,NaN,NaN,NaN
2,635869,Al Wehda Club,Al-Qadisiyah FC,2,1,King Abdulaziz Sports City Stadium,Khaled Salwi,1,2020-10-17T20:00:00+03:00,2020/2021,Match Finished,NaN,NaN,NaN,NaN,NaN,NaN
3,635870,Al-Hilal Saudi FC,Al-Ain,1,0,Prince Faisal bin Fahd Stadium,Sultan Al-Harbi,1,2020-10-17T21:15:00+03:00,2020/2021,Match Finished,NaN,NaN,NaN,NaN,NaN,NaN
4,635871,Al Baten,Al-Ahli Jeddah,0,1,Al-Batin FC Stadium,Khaled Al Teris,1,2020-10-18T18:00:00+03:00,2020/2021,Match Finished,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,657878,Al-Raed,Al-Fateh,None,None,King Abdullah bin Abdulaziz City,None,30,2021-05-30T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN
236,657879,Al Shabab,Al Wehda Club,None,None,Al-Shabab Club Stadium,None,30,2021-05-30T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN
237,657880,Al-Ahli Jeddah,Al-Ettifaq,None,None,King Abdullah Sports City,None,30,2021-05-30T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN
238,657881,Al-Hilal Saudi FC,Al-Faisaly FC,None,None,Prince Faisal bin Fahd Stadium,None,30,2021-05-30T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
season2020 = addMatchDetails(season2020, 0, 30)

In [25]:
season2020

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,round,date,season,status,home_coach,home_foramtion,home_players,away_coach,away_formation,away_players
0,635867,Al Shabab,Abha,1,0,Prince Faisal bin Fahd Stadium,Mohammed Al Hoaish,1,2020-10-17T18:00:00+03:00,2020/2021,Match Finished,None,4-4-2,"[(Giedrius Arlauskis, G), (Mohammed Salem, D),...",A. Chebbi,4-2-3-1,"[(Abdelali Mhamdi, G), (Saeed Ali, D), (Ahmad ..."
1,635868,Al Taawon,Al-Faisaly FC,1,1,King Abdullah bin Abdulaziz City,Faisal Alblwi,1,2020-10-17T18:10:00+03:00,2020/2021,Match Finished,Abdullah Assiri,5-4-1,"[(Cássio, G), (Faisal Darwish, D), (Ahmad Asir...",Péricles Chamusca,4-3-3,"[(Mustafa Malaika, G), (Khaled Daghriri, D), (..."
2,635869,Al Wehda Club,Al-Qadisiyah FC,2,1,King Abdulaziz Sports City Stadium,Khaled Salwi,1,2020-10-17T20:00:00+03:00,2020/2021,Match Finished,None,4-4-2,"[(Abdullah Hussein Al Owisher, G), (Hamad Al J...",None,4-3-2-1,"[(Faisal Al-Masrahi, G), (Hamad Al Tohaifan, D..."
3,635870,Al-Hilal Saudi FC,Al-Ain,1,0,Prince Faisal bin Fahd Stadium,Sultan Al-Harbi,1,2020-10-17T21:15:00+03:00,2020/2021,Match Finished,None,4-4-2,"[(Abdullah Al-Mayouf, G), (Amiri Kurdi, D), (M...",None,4-2-3-1,"[(Mohammed Mazyad Alshammari, G), (Hassan Al-H..."
4,635871,Al Baten,Al-Ahli Jeddah,0,1,Al-Batin FC Stadium,Khaled Al Teris,1,2020-10-18T18:00:00+03:00,2020/2021,Match Finished,José Garrido,4-3-2-1,"[(Mazyad Al Enazi, G), (Saad Al Khayri, D), (R...",V. Milojević,4-2-3-1,"[(Yasser Al-Mosailem, G), (Hassoun, D), (Moham..."
5,635872,Al-Nassr,Al-Fateh,1,2,Prince Faisal bin Fahd Stadium,Majed Al Shamrani,1,2020-10-18T18:00:00+03:00,2020/2021,Match Finished,Rui Vitória,4-2-3-1,"[(Brad Jones, G), (Osama Al Khalaf, D), (Maico...",Y. Ferrera,4-4-2,"[(Maksym Koval, G), (Nawaf Boushal, D), (Amaar..."
6,635873,Dhamk,Al-Raed,2,3,Prince Sultan bin Abdulaziz Sports City Stadium,Shukri Al Hanfosh,1,2020-10-18T18:20:00+03:00,2020/2021,Match Finished,N. Zekri,4-2-3-1,"[(Moustapha Zeghba, G), (Tariq Mohammed, D), (...",B. Hasi,4-3-3,"[(Ahmad Al Harbi, G), (Mohammed Al-Dosari, D),..."
7,635874,Al-Ittihad FC,Al-Ettifaq,1,2,King Abdullah Sports City,Turki Al Khudair,1,2020-10-18T20:00:00+03:00,2020/2021,Match Finished,Fábio Carille,4-4-2,"[(Marcelo Grohe, G), (Saud Abdulhamid, D), (Zi...",Khaled Al Atawi,4-4-2,"[(Raïs M'Bolhi, G), (Saleh Al Qumayzi, D), (Sa..."
8,635875,Abha,Al-Hilal Saudi FC,1,1,Prince Sultan bin Abdulaziz Sports City Stadium,Majed Al Shamrani,2,2020-10-22T15:40:00+03:00,2020/2021,Match Finished,A. Chebbi,4-2-3-1,"[(Abdelali Mhamdi, G), (Saeed Al Hamsal, D), (...",R. Lucescu,4-2-3-1,"[(Abdullah Al-Mayouf, G), (Mohammed Al-Burayk,..."
9,635876,Al Taawon,Al-Nassr,1,0,King Abdullah bin Abdulaziz City,Mohammed Al Hoaish,2,2020-10-22T18:05:00+03:00,2020/2021,Match Finished,Abdullah Assiri,4-2-3-1,"[(Cássio, G), (Yaseen Barnawi, D), (Ahmad Asir...",Rui Vitória,4-2-3-1,"[(Brad Jones, G), (Jin-su Kim, D), (Abdulelah ..."


In [95]:
len(season2018["home_team"].unique())

16

In [96]:
len(season2018["away_team"].unique())

16

In [105]:
season2018.isnull().sum()

fixture_id        0
home_team         0
away_team         0
home_goals        0
away_goals        0
venue             0
referee           9
date              0
season            0
home_coach        0
away_coach        0
home_formation    0
away_formation    0
home_players      0
away_players      0
round             0
status            0
dtype: int64

In [37]:
season2019.isnull().sum()

fixture_id         0
home_team          0
away_team          0
home_goals         0
away_goals         0
venue              0
referee           31
date               0
season             0
home_coach         0
away_coach         0
home_formation    17
away_formation    17
home_players       0
away_players       0
round              0
status             0
H_avg_ratings     17
A_avg_ratings     17
dtype: int64

In [14]:
season2019.loc[(season2019["A_avg_ratings"].isnull()) & (season2019["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
186,312919,Al Wehda Club,Al-Raed,0.0,0.0,King Abdul Aziz Stadium,NaN,2020-08-09T19:30:00+03:00,2019/2020,D. Carreño,B. Hasi,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('A. Doukha', None), ('Hussein Al Shuwaish', ...",24,Match Finished,NaN,NaN
199,312927,Al-Nassr,Al Wehda Club,1.0,0.0,King Fahd International Stadium,P. Kralovec,2020-08-15T20:50:00+03:00,2019/2020,Rui Vitória,D. Carreño,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Sulta...","[('Abdullah Al Jadani', None), ('Botía', None)...",25,Match Finished,NaN,NaN
204,312934,Al-Adalah,Al Wehda Club,2.0,4.0,Prince Abdullah bin Jalawi Sports City Stadium,M. Conger,2020-08-20T18:50:00+03:00,2019/2020,G. Solinas,Essa Al Mehyani,NaN,NaN,"[('Ali Al Mazidi', None), ('Mortada Al Barrih'...","[('Abdullah Al Jadani', None), ('Botía', None)...",26,Match Finished,NaN,NaN
212,312945,Al-Hilal Saudi FC,Al-Faisaly FC,2.0,0.0,King Saud University Stadium,NaN,2020-08-25T19:00:00+03:00,2019/2020,R. Lucescu,Péricles Chamusca,NaN,NaN,"[('Abdullah Al Maiouf', None), ('Yasir Al Shah...","[('Mustafa Malayekah', None), ('Igor Rossi', N...",27,Match Finished,NaN,NaN
213,312943,Dhamk,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdul Aziz Stadium,NaN,2020-08-25T19:05:00+03:00,2019/2020,N. Zekri,V. Milojević,NaN,NaN,"[('M. Zeghba', None), ('S. Vittor', None), ('M...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",27,Match Finished,NaN,NaN
214,312944,Al Wehda Club,Al Taawon,2.0,1.0,King Abdul Aziz Stadium,NaN,2020-08-25T19:20:00+03:00,2019/2020,Essa Al Mehyani,Vítor Campelos,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('Cássio', None), ('Ricardo Machado', None), ...",27,Match Finished,NaN,NaN
215,312942,Al-Nassr,Al-Adalah,6.0,1.0,King Fahd International Stadium,A. Treimanis,2020-08-25T20:50:00+03:00,2019/2020,Rui Vitória,G. Solinas,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Abdul...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",27,Match Finished,NaN,NaN
221,312950,Al-Faisaly FC,Al Wehda Club,2.0,1.0,Al Majma'ah Sports City Stadium,João Pedro Pinheiro,2020-08-30T18:55:00+03:00,2019/2020,Péricles Chamusca,Essa Al Mehyani,NaN,NaN,"[('Mustafa Malayekah', None), ('Igor Rossi', N...","[('Abdullah Al Jadani', None), ('Botía', None)...",28,Match Finished,NaN,NaN
224,312957,Abha,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdulaziz Sports City Stadium,Khaled Al Teris,2020-09-04T20:35:00+03:00,2019/2020,A. Chebbi,V. Milojević,NaN,NaN,"[(""A. M'Hamdi"", None), ('Osama Ashoor', None),...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",29,Match Finished,NaN,NaN
225,312958,Al-Fayha,Al-Adalah,2.0,0.0,Al Majma'ah Sports City Stadium,M. Conger,2020-09-04T20:35:00+03:00,2019/2020,Jorge Simão,G. Solinas,NaN,NaN,"[('Moslem Freej', None), ('Nawaf Al Sabhi', No...","[('Ali Al Mazidi', None), ('Mortada Al Barrih'...",29,Match Finished,NaN,NaN


# Getting the players ratings

In [20]:

for i in range(204, 240):
    fixture_id = season2018.loc[i, "fixture_id"]
    home_team = season2018.loc[i, "home_team"]
    
    url = f"https://api-football-v1.p.rapidapi.com/v2/players/fixture/{fixture_id}"

    headers = {
        'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
        'x-rapidapi-key': my_api_key
        }

    response = requests.request("GET", url, headers=headers)

    response.json()
    
    players = response.json()["api"]["players"]

    Hteam_avg_ratings = []
    Ateam_avg_ratings = []
    
    for player in players:
        
        if player["team_name"] == home_team:
            try:
                Hteam_avg_ratings.append(float(player["rating"]))
            except:
                pass
        else:
            try:
                Ateam_avg_ratings.append(float(player["rating"]))
            except:
                pass

    Hteam_avg_ratings = round(sum(Hteam_avg_ratings) / len(Hteam_avg_ratings), 2)
    Ateam_avg_ratings = round(sum(Ateam_avg_ratings) / len(Ateam_avg_ratings), 2)
    
    season2018.loc[i, "H_avg_ratings"] = Hteam_avg_ratings
    season2018.loc[i, "A_avg_ratings"] = Ateam_avg_ratings


NameError: name 'season2018' is not defined

### We repeat the same process but for the 2019/2020 season

In [99]:
season2020.columns

Index(['fixture_id', 'home_team', 'away_team', 'home_goals', 'away_goals',
       'venue', 'referee', 'round', 'date', 'season', 'status', 'home_coach',
       'home_foramtion', 'home_players', 'away_coach', 'away_formation',
       'away_players'],
      dtype='object')

In [113]:
season2020["H_avg_ratings"] = np.nan
season2020["A_avg_ratings"] = np.nan 

In [34]:
for i in range(240):
    fixture_id = season2019.loc[i, "fixture_id"]
    home_team = season2019.loc[i, "home_team"]
    
    try:

        url = f"https://api-football-v1.p.rapidapi.com/v2/players/fixture/{fixture_id}"

        headers = {
            'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
            'x-rapidapi-key': my_api_key
            }

        response = requests.request("GET", url, headers=headers)

        response.json()

        players = response.json()["api"]["players"]

        Hteam_avg_ratings = []
        Ateam_avg_ratings = []

        for player in players:

            if player["team_name"] == home_team:
                try:
                    Hteam_avg_ratings.append(float(player["rating"]))
                except:
                    pass
            else:
                try:
                    Ateam_avg_ratings.append(float(player["rating"]))
                except:
                    pass

        Hteam_avg_ratings = round(sum(Hteam_avg_ratings) / len(Hteam_avg_ratings), 2)
        Ateam_avg_ratings = round(sum(Ateam_avg_ratings) / len(Ateam_avg_ratings), 2)

        season2019.loc[i, "H_avg_ratings"] = Hteam_avg_ratings
        season2019.loc[i, "A_avg_ratings"] = Ateam_avg_ratings
    except:
        pass

In [35]:
season2019.loc[(season2019["H_avg_ratings"].isnull()) & (season2019["status"] == "Match Finished")]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
186,312919,Al Wehda Club,Al-Raed,0.0,0.0,King Abdul Aziz Stadium,NaN,2020-08-09T19:30:00+03:00,2019/2020,D. Carreño,B. Hasi,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('A. Doukha', None), ('Hussein Al Shuwaish', ...",24,Match Finished,NaN,NaN
199,312927,Al-Nassr,Al Wehda Club,1.0,0.0,King Fahd International Stadium,P. Kralovec,2020-08-15T20:50:00+03:00,2019/2020,Rui Vitória,D. Carreño,NaN,NaN,"[('B. Jones', None), ('Maicon', None), ('Sulta...","[('Abdullah Al Jadani', None), ('Botía', None)...",25,Match Finished,NaN,NaN
204,312934,Al-Adalah,Al Wehda Club,2.0,4.0,Prince Abdullah bin Jalawi Sports City Stadium,M. Conger,2020-08-20T18:50:00+03:00,2019/2020,G. Solinas,Essa Al Mehyani,NaN,NaN,"[('Ali Al Mazidi', None), ('Mortada Al Barrih'...","[('Abdullah Al Jadani', None), ('Botía', None)...",26,Match Finished,NaN,NaN
212,312945,Al-Hilal Saudi FC,Al-Faisaly FC,2.0,0.0,King Saud University Stadium,NaN,2020-08-25T19:00:00+03:00,2019/2020,R. Lucescu,Péricles Chamusca,NaN,NaN,"[('Abdullah Al Maiouf', None), ('Yasir Al Shah...","[('Mustafa Malayekah', None), ('Igor Rossi', N...",27,Match Finished,NaN,NaN
213,312943,Dhamk,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdul Aziz Stadium,NaN,2020-08-25T19:05:00+03:00,2019/2020,N. Zekri,V. Milojević,NaN,NaN,"[('M. Zeghba', None), ('S. Vittor', None), ('M...","[('Yasser Al Mosailem', None), ('Lucas Lima', ...",27,Match Finished,NaN,NaN
214,312944,Al Wehda Club,Al Taawon,2.0,1.0,King Abdul Aziz Stadium,NaN,2020-08-25T19:20:00+03:00,2019/2020,Essa Al Mehyani,Vítor Campelos,NaN,NaN,"[('Abdullah Al Jadani', None), ('Botía', None)...","[('Cássio', None), ('Ricardo Machado', None), ...",27,Match Finished,NaN,NaN
215,312942,Al-Nassr,Al-Adalah,6.0,1.0,King Fahd International Stadium,A. Treimanis,2020-08-25T20:50:00+03:00,2019/2020,Rui Vitória,G. Solinas,None,None,"[(B. Jones, None), (Maicon, None), (Abdulrahma...","[(Ali Al Mazidi, None), (Mortada Al Barrih, No...",27,Match Finished,NaN,NaN
221,312950,Al-Faisaly FC,Al Wehda Club,2.0,1.0,Al Majma'ah Sports City Stadium,João Pedro Pinheiro,2020-08-30T18:55:00+03:00,2019/2020,Péricles Chamusca,Essa Al Mehyani,None,None,"[(Mustafa Malayekah, None), (Igor Rossi, None)...","[(Abdullah Al Jadani, None), (Botía, None), (R...",28,Match Finished,NaN,NaN
224,312957,Abha,Al-Ahli Jeddah,2.0,1.0,Prince Sultan bin Abdulaziz Sports City Stadium,Khaled Al Teris,2020-09-04T20:35:00+03:00,2019/2020,A. Chebbi,V. Milojević,None,None,"[(A. M'Hamdi, None), (Osama Ashoor, None), (M....","[(Yasser Al Mosailem, None), (Lucas Lima, None...",29,Match Finished,NaN,NaN
225,312958,Al-Fayha,Al-Adalah,2.0,0.0,Al Majma'ah Sports City Stadium,M. Conger,2020-09-04T20:35:00+03:00,2019/2020,Jorge Simão,G. Solinas,None,None,"[(Moslem Freej, None), (Nawaf Al Sabhi, None),...","[(Ali Al Mazidi, None), (Mortada Al Barrih, No...",29,Match Finished,NaN,NaN


In [40]:
season2019.loc[236]

fixture_id                                                   312969
home_team                                            Al-Ahli Jeddah
away_team                                                   Al-Raed
home_goals                                                        2
away_goals                                                        0
venue                                     King Abdullah Sports City
referee                                                        None
date                                      2020-09-09T20:35:00+03:00
season                                                    2019/2020
home_coach                                             V. Milojević
away_coach                                                  B. Hasi
home_formation                                              4-1-4-1
away_formation                                              4-2-3-1
home_players      [(Mohammed Al-Owais, G), (Hassoun, D), (Motaz ...
away_players      [(Azzedine Doukha, G), (Mohamm

In [41]:
season2019.head()

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
0,234777,Al-Nassr,Dhamk,2.0,0.0,King Fahd International Stadium,M. Conger,2019-08-22T19:00:00+03:00,2019/2020,Rui Vitória,M. Kouki,4-2-3-1,4-3-3,"[('B. Jones', 'G'), ('Maicon', 'D'), ('Abdulra...","[('C. Rahmani', 'G'), ('Jorge Fellipe', 'D'), ...",1,Match Finished,7.27,6.63
1,234778,Al-Ahli Jeddah,Al-Adalah,1.0,1.0,King Abdullah Sports City,B. Gassama,2019-08-22T20:50:00+03:00,2019/2020,B. Ivanković,S. Kasri,4-4-2,4-3-3,"[('Mohammed Al-Owais', 'G'), ('Hussein Abdul G...","[('A. Mathlouthi', 'G'), ('A. Ali Guechi', 'D'...",1,Match Finished,6.97,6.84
2,234779,Al-Hilal Saudi FC,Abha,4.0,2.0,King Saud University Stadium,D. Higler,2019-08-23T19:00:00+03:00,2019/2020,R. Lucescu,A. Chebbi,4-2-3-1,4-2-3-1,"[('Abdullah Al Maiouf', 'G'), ('Mohammed Jahfa...","[(""A. M'Hamdi"", 'G'), ('Karam Barnawi', 'D'), ...",1,Match Finished,7.01,6.51
3,234780,Al Taawon,Al-Hazm,1.0,1.0,King Abdullah Sport City Stadium,Luis Godinho,2019-08-23T19:10:00+03:00,2019/2020,Paulo Sérgio,D. Isăilă,4-4-2,4-2-3-1,"[('Cássio', 'G'), ('Ricardo Machado', 'D'), ('...","[('M. Asselah', 'G'), ('Naif Al-Mousa', 'D'), ...",1,Match Finished,6.58,6.76
4,234781,Al-Ittihad FC,Al-Raed,3.0,1.0,King Abdullah Sports City,S. Moen,2019-08-23T20:50:00+03:00,2019/2020,J. Sierra,B. Hasi,4-3-3,4-3-3,"[('Fawaz Al Qarni', 'G'), ('M. da Costa', 'D')...","[('A. Doukha', 'G'), ('Hussein Al Shuwaish', '...",1,Match Finished,7.01,6.61


In [42]:
season2019.loc[season2019["status"] == "Match Finished"].tail()

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
235,312968,Al-Faisaly FC,Abha,2.0,1.0,Al Majma'ah Sports City Stadium,None,2020-09-09T20:35:00+03:00,2019/2020,Péricles Chamusca,A. Chebbi,4-2-3-1,4-2-3-1,"[(Mustafa Malaika, G), (Khaled Al-Ghamdi, D), ...","[(Abdelali Mhamdi, G), (Osama Ashoor, D), (Meh...",30,Match Finished,6.68,6.49
236,312969,Al-Ahli Jeddah,Al-Raed,2.0,0.0,King Abdullah Sports City,None,2020-09-09T20:35:00+03:00,2019/2020,V. Milojević,B. Hasi,4-1-4-1,4-2-3-1,"[(Mohammed Al-Owais, G), (Hassoun, D), (Motaz ...","[(Azzedine Doukha, G), (Mohammed Al-Dosari, D)...",30,Match Finished,7.10,6.51
237,312970,Al Taawon,Al-Fayha,1.0,0.0,King Abdullah Sport City Stadium,None,2020-09-09T20:35:00+03:00,2019/2020,Vítor Campelos,Jorge Simão,4-2-3-1,4-4-2,"[(Cássio, G), (Yaseen Barnawi, D), (Ricardo Ma...","[(Moslem Al-Freej, G), (Mokher Al-Rashidi, D),...",30,Match Finished,6.96,6.56
238,312971,Al-Hilal Saudi FC,Al Shabab,2.0,1.0,King Saud University Stadium,None,2020-09-09T20:35:00+03:00,2019/2020,R. Lucescu,Luis García,4-2-3-1,4-2-3-1,"[(Abdullah Al-Mayouf, G), (Mohammed Al-Burayk,...","[(Marwan Al-Haidari, G), (Hassan Fallatah, D),...",30,Match Finished,6.83,7.01
239,312972,Al-Nassr,Al-Ettifaq,3.0,2.0,Prince Faisal bin Fahd Stadium,None,2020-09-09T20:35:00+03:00,2019/2020,Rui Vitória,Khaled Al Atawi,5-4-1,4-4-2,"[(Brad Jones, G), (Sultan Al-Ghannam, D), (Abd...","[(Raïs M'Bolhi, G), (Saleh Al Qumayzi, D), (Sa...",30,Match Finished,6.68,6.66


In [43]:
season2019.to_csv("season2019 v9.csv")

In [16]:
season2019.tail()

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,date,season,home_coach,away_coach,home_formation,away_formation,home_players,away_players,round,status,H_avg_ratings,A_avg_ratings
235,312968,Al-Faisaly FC,Abha,2.0,1.0,Al Majma'ah Sports City Stadium,NaN,2020-09-09T20:35:00+03:00,2019/2020,Péricles Chamusca,A. Chebbi,4-2-3-1,4-2-3-1,"[('Mustafa Malaika', 'G'), ('Khaled Al-Ghamdi'...","[('Abdelali Mhamdi', 'G'), ('Osama Ashoor', 'D...",30,Match Finished,6.68,6.49
236,312969,Al-Ahli Jeddah,Al-Raed,2.0,0.0,King Abdullah Sports City,NaN,2020-09-09T20:35:00+03:00,2019/2020,V. Milojević,B. Hasi,4-1-4-1,4-2-3-1,"[('Mohammed Al-Owais', 'G'), ('Hassoun', 'D'),...","[('Azzedine Doukha', 'G'), ('Mohammed Al-Dosar...",30,Match Finished,7.10,6.51
237,312970,Al Taawon,Al-Fayha,1.0,0.0,King Abdullah Sport City Stadium,NaN,2020-09-09T20:35:00+03:00,2019/2020,Vítor Campelos,Jorge Simão,4-2-3-1,4-4-2,"[('Cássio', 'G'), ('Yaseen Barnawi', 'D'), ('R...","[('Moslem Al-Freej', 'G'), ('Mokher Al-Rashidi...",30,Match Finished,6.96,6.56
238,312971,Al-Hilal Saudi FC,Al Shabab,2.0,1.0,King Saud University Stadium,NaN,2020-09-09T20:35:00+03:00,2019/2020,R. Lucescu,Luis García,4-2-3-1,4-2-3-1,"[('Abdullah Al-Mayouf', 'G'), ('Mohammed Al-Bu...","[('Marwan Al-Haidari', 'G'), ('Hassan Fallatah...",30,Match Finished,6.83,7.01
239,312972,Al-Nassr,Al-Ettifaq,3.0,2.0,Prince Faisal bin Fahd Stadium,NaN,2020-09-09T20:35:00+03:00,2019/2020,Rui Vitória,Khaled Al Atawi,5-4-1,4-4-2,"[('Brad Jones', 'G'), ('Sultan Al-Ghannam', 'D...","[(""Raïs M'Bolhi"", 'G'), ('Saleh Al Qumayzi', '...",30,Match Finished,6.68,6.66


In [56]:
season2020.to_csv("season_2020 30 matches.csv")

In [103]:
season2020 = pd.read_csv("Season_2020 30 matches.csv").drop(columns=["Unnamed: 0"])

In [104]:
season2020

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,round,date,season,status,home_coach,home_foramtion,home_players,away_coach,away_formation,away_players
0,635867,Al Shabab,Abha,1.0,0.0,Prince Faisal bin Fahd Stadium,Mohammed Al Hoaish,1,2020-10-17T18:00:00+03:00,2020/2021,Match Finished,NaN,4-4-2,"[('Giedrius Arlauskis', 'G'), ('Mohammed Salem...",A. Chebbi,4-2-3-1,"[('Abdelali Mhamdi', 'G'), ('Saeed Ali', 'D'),..."
1,635868,Al Taawon,Al-Faisaly FC,1.0,1.0,King Abdullah bin Abdulaziz City,Faisal Alblwi,1,2020-10-17T18:10:00+03:00,2020/2021,Match Finished,Abdullah Assiri,5-4-1,"[('Cássio', 'G'), ('Faisal Darwish', 'D'), ('A...",Péricles Chamusca,4-3-3,"[('Mustafa Malaika', 'G'), ('Khaled Daghriri',..."
2,635869,Al Wehda Club,Al-Qadisiyah FC,2.0,1.0,King Abdulaziz Sports City Stadium,Khaled Salwi,1,2020-10-17T20:00:00+03:00,2020/2021,Match Finished,NaN,4-4-2,"[('Abdullah Hussein Al Owisher', 'G'), ('Hamad...",NaN,4-3-2-1,"[('Faisal Al-Masrahi', 'G'), ('Hamad Al Tohaif..."
3,635870,Al-Hilal Saudi FC,Al-Ain,1.0,0.0,Prince Faisal bin Fahd Stadium,Sultan Al-Harbi,1,2020-10-17T21:15:00+03:00,2020/2021,Match Finished,NaN,4-4-2,"[('Abdullah Al-Mayouf', 'G'), ('Amiri Kurdi', ...",NaN,4-2-3-1,"[('Mohammed Mazyad Alshammari', 'G'), ('Hassan..."
4,635871,Al Baten,Al-Ahli Jeddah,0.0,1.0,Al-Batin FC Stadium,Khaled Al Teris,1,2020-10-18T18:00:00+03:00,2020/2021,Match Finished,José Garrido,4-3-2-1,"[('Mazyad Al Enazi', 'G'), ('Saad Al Khayri', ...",V. Milojević,4-2-3-1,"[('Yasser Al-Mosailem', 'G'), ('Hassoun', 'D')..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,657878,Al-Raed,Al-Fateh,NaN,NaN,King Abdullah bin Abdulaziz City,NaN,30,2021-05-30T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN
236,657879,Al Shabab,Al Wehda Club,NaN,NaN,Al-Shabab Club Stadium,NaN,30,2021-05-30T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN
237,657880,Al-Ahli Jeddah,Al-Ettifaq,NaN,NaN,King Abdullah Sports City,NaN,30,2021-05-30T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN
238,657881,Al-Hilal Saudi FC,Al-Faisaly FC,NaN,NaN,Prince Faisal bin Fahd Stadium,NaN,30,2021-05-30T21:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
season2020 = update_df(season2020, season2020code)

In [160]:
for i in range(154, 176):
    season2020 = addMatchDetails(season2020, i)

In [161]:
season2020[150:160]

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,round,date,season,status,home_coach,home_foramtion,home_players,away_coach,away_formation,away_players,H_avg_ratings,A_avg_ratings
150,657793,Al-Qadisiyah FC,Al-Nassr,1.0,0.0,Prince Saud bin Jalawi Stadium,Faisal Alblwi,19,2021-02-18T17:10:00+03:00,2020/2021,Match Finished,Y. Al Mannai,4-4-1-1,"[(Faisal Al-Masrahi, G), (Hamad Al Yami, D), (...",A. Horvat,4-2-3-1,"[(Brad Jones, G), (Osama Al Khalaf, D), (Ali A...",NaN,NaN
151,657794,Al-Hilal Saudi FC,Al-Ettifaq,3.0,1.0,Prince Faisal bin Fahd Stadium,M. Ndiaye,19,2021-02-18T19:20:00+03:00,2020/2021,Match Finished,None,4-3-3,"[(Habib Al-Wutaian, G), (Mohammed Al-Burayk, D...",None,4-4-2,"[(Mohammed Alhaeti, G), (Saleh Al Qumayzi, D),...",NaN,NaN
152,657795,Abha,Al-Raed,0.0,3.0,Prince Sultan bin Abdulaziz Sports City Stadium,Sultan Al-Harbi,20,2021-02-22T15:10:00+03:00,2020/2021,Match Finished,A. Chebbi,4-2-3-1,"[(Abdelali Mhamdi, G), (Nader Abdullah Al Shar...",B. Hasi,3-4-2-1,"[(Azzedine Doukha, G), (Abdullah Al-Fahad, D),...",NaN,NaN
153,657796,Al Wehda Club,Al Taawon,0.0,2.0,King Abdulaziz Sports City Stadium,Faisal Alblwi,20,2021-02-22T18:00:00+03:00,2020/2021,Match Finished,Mahmoud Al-Hadid,4-2-3-1,"[(Abdullah Hussein Al Owisher, G), (Hamad Al J...",Abdullah Assiri,4-2-3-1,"[(Cássio, G), (Mohamed Al-Ghamdi, D), (Iago Sa...",NaN,NaN
154,657797,Al Shabab,Al-Ahli Jeddah,3.0,0.0,Al-Shabab Club Stadium,Majed Al Shamrani,20,2021-02-22T19:35:00+03:00,2020/2021,Match Finished,None,4-2-3-1,"[(Zaid Al-Bawardi, G), (Mohammed Salem, D), (I...",None,4-1-4-1,"[(Mohammed Al-Owais, G), (Yazeed Al Bakr, D), ...",NaN,NaN
155,657798,Al Baten,Al-Qadisiyah FC,2.0,2.0,Al-Batin FC Stadium,Shukri Al Hanfosh,20,2021-02-23T14:45:00+03:00,2020/2021,Match Finished,José Garrido,4-2-3-1,"[(Martín Campaña, G), (Al-Aowda, D), (Massod A...",Y. Al Mannai,4-4-1-1,"[(Faisal Al-Masrahi, G), (Anas Zabani, D), (Ur...",NaN,NaN
156,657799,Al-Ain,Dhamk,1.0,1.0,King Saud Sport City Stadium,Mohammed Al Hoaish,20,2021-02-23T15:10:00+03:00,2020/2021,Match Finished,Pablo Machín,3-4-2-1,"[(Ameen Bokhari, G), (Hassan Al-Harbi, D), (Ba...",K. Režić,4-1-4-1,"[(Moustapha Zeghba, G), (Arif Al-Haydar, D), (...",NaN,NaN
157,657800,Al-Fateh,Al-Ettifaq,2.0,3.0,Prince Abdullah bin Jalawi Sports City Stadium,Turki Al Khudair,20,2021-02-23T17:20:00+03:00,2020/2021,Match Finished,Y. Ferrera,4-2-3-1,"[(Maksym Koval, G), (Nawaf Boushal, D), (Amaar...",Khaled Al Atawi,4-4-2,"[(Mohammed Alhaeti, G), (Saleh Al Qumayzi, D),...",NaN,NaN
158,657801,Al-Ittihad FC,Al-Faisaly FC,1.0,3.0,King Abdulaziz Sports City Stadium,Khaled Al Teris,20,2021-02-23T18:00:00+03:00,2020/2021,Match Finished,Fábio Carille,4-2-3-1,"[(Marcelo Grohe, G), (Saud Abdulhamid, D), (Zi...",Péricles Chamusca,4-2-3-1,"[(Mustafa Malaika, G), (Abdullah Alhassan, D),...",NaN,NaN
159,657802,Al-Nassr,Al-Hilal Saudi FC,1.0,0.0,Mrsool Park,E. Ostojich,20,2021-02-23T19:35:00+03:00,2020/2021,Match Finished,None,4-1-4-1,"[(Brad Jones, G), (Sultan Al-Ghannam, D), (Abd...",None,4-2-3-1,"[(Abdullah Al-Mayouf, G), (Mohammed Al-Burayk,...",NaN,NaN


In [145]:
season2020.to_csv("Season2020 v3 175 Matches.csv")

In [156]:
season2020.query("status == 'Not Started'")

,fixture_id,home_team,away_team,home_goals,away_goals,venue,referee,round,date,season,status,home_coach,home_foramtion,home_players,away_coach,away_formation,away_players,H_avg_ratings,A_avg_ratings
185,657833,Al-Raed,Al-Ittihad FC,NaN,NaN,King Abdullah bin Abdulaziz City,NaN,24,2021-03-20T16:15:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,657827,Al-Ettifaq,Al-Faisaly FC,NaN,NaN,Prince Mohamed bin Fahd Stadium,NaN,24,2021-03-20T18:30:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,657832,Al-Fateh,Al-Ahli Jeddah,NaN,NaN,Prince Abdullah bin Jalawi Sports City Stadium,NaN,24,2021-03-20T18:30:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,657828,Al Shabab,Al Taawon,NaN,NaN,Al-Shabab Club Stadium,NaN,24,2021-03-20T18:40:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,657829,Al-Hilal Saudi FC,Al-Qadisiyah FC,NaN,NaN,Prince Faisal bin Fahd Stadium,NaN,24,2021-03-20T20:45:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,657834,Al-Nassr,Al Baten,NaN,NaN,Mrsool Park,NaN,24,2021-03-20T20:45:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,657830,Abha,Dhamk,NaN,NaN,Prince Sultan bin Abdulaziz Sports City Stadium,NaN,24,2021-03-21T16:05:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,657831,Al-Ain,Al Wehda Club,NaN,NaN,King Saud Sport City Stadium,NaN,24,2021-03-21T16:10:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,657836,Al-Ahli Jeddah,Al-Raed,NaN,NaN,King Abdullah Sports City,NaN,25,2021-04-08T20:50:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,657837,Al Taawon,Al-Ain,NaN,NaN,King Abdullah bin Abdulaziz City,NaN,25,2021-04-09T19:00:00+03:00,2020/2021,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
